In [ ]:
!pip install -qU \
    nemoguardrails==0.4.0 \
    openai==0.27.8

In [ ]:
import os
#
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
colang_content = """
# define limits
define user ask weather
    "how is the weather today?"
    "idk if i should wear boots today is it raining?",
    "what is the weather like today?,
    "what is the weather like in london?",

define bot answer weather
    bot report weather

define flow weather
    user ask weather
    $coords = execute location_api()
    $weather = execute weather_api(coords=$coords)
    bot answer weather

# here we use the chatbot for anything else
define flow
    user ...
    bot greeting
"""
yaml_content = """
models:
- type: main
  engine: openai
  model: gpt-4o-mini
"""

In [ ]:
import requests

async def weather_api(coords: list):
    latitude, longitude = coords
    res = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={
            "latitude": latitude,
            "longitude": longitude,
            "current_weather": "true"
        }
    )
    weather = res.json()["current_weather"]
    weather_report = f"""The current weather is:
    temperature: {weather["temperature"]}
    windspeed: {weather["windspeed"]}
    wind direction: {weather["winddirection"]} degrees
    And it is {"daytime" if weather["is_day"] else "nightime"}"""
    return weather_report

async def location_api():
    res = requests.get("http://ip-api.com/json/")
    return res.json()['lat'], res.json()['lon']

We've defined our location and weather APIs required for our bot to give us up to date advice on the weather in our current location.

In [ ]:
from nemoguardrails import LLMRails, RailsConfig

# initialize rails config
config = RailsConfig.from_content(
    colang_content=colang_content,
    yaml_content=yaml_content
)
# create rails
rails = LLMRails(config, verbose=True)

rails.register_action(action=location_api, name="location_api")
rails.register_action(action=weather_api, name="weather_api")

In [ ]:
await rails.generate_async(prompt="hello")

In [ ]:
await rails.generate_async(prompt="how is the weather?")